<a href="https://colab.research.google.com/github/ishanbhattacharjee12/Next-Word-Predictor-DL-Project/blob/main/pytorch_lstm_next_word_predictor_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [3]:
document = """About Our Voyages
What is the travel package for the Martian Colony Initiative (MCI 2025)?
The initiative uses a monthly preparation plan where you make payments of $899/month to cover training and logistics.

What is the total duration of the pre-flight program?
The total duration of the pre-flight training and preparation is 8 months. Therefore, the total preparation fee becomes 899*8 = $7192 (approx.).

What is the syllabus of the astronaut preparation program?
We will be covering the following modules:

Zero-Gravity Acclimatization

Exo-geology and Planetary Science

Hydroponics and Sustainable Farming

SQL for Celestial Body Cataloging

Orbital Mechanics and Propulsion Theory

Spacesuit Maintenance Algorithms

Practical Rover Operation

Emergency Protocols

Case studies of past missions
You can check the detailed syllabus here - https://starlight-voyages.com/courses/Starlight-Martian-Colony-Initiative-848449bfe4b0615a1bbed401

Will training for the Outer Planets be a part of this program?
No, Jupiter and Saturn missions are not a part of this program’s curriculum. This is strictly focused on Mars.

What if I miss a live simulation? Will I get a recording of it?
Yes, all our theoretical sessions are recorded, so even if you miss a session you can go back and watch the recording. Practical simulations must be rescheduled.

Where can I find the training schedule?
Check out this data sheet to see the month-by-month timetable of the program - https://docs.google.com/spreadsheets/d/27PpTax_B7ORBeDg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.

What is the time duration of all the live training sessions?
Roughly, all sessions last 3 hours.

What is the language spoken by the commander during the sessions?
English.

How will I be informed about the upcoming training module?
You will get a priority communication from our side before every module once you become a registered candidate.

Can I do this program if I am from a non-science background?
Yes, absolutely.

I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.

If I join/pay in the middle, will I be able to see all the past simulations?
Yes, once you make the payment you will be able to see all the past theoretical content in your flight dashboard.

Where do I have to submit my completed tasks?
You don’t have to submit the tasks. We will provide you with the solution parameters, you have to self-evaluate your performance.

Will we do mission case studies in the program?
Yes.

Where can we contact you?
You can send a transmission to mission.control@starlight-voyages.com.

Payment/Registration Related Questions
Where do we have to make our payments? On a partner site or the main website?
You have to make all your monthly payments on our official website. Here is the link for our website - https://starlight-voyages.com/

Can we pay the entire amount of $7192 all at once?
Unfortunately no, the program follows a monthly preparation model.

What is the validity of a monthly subscription? Suppose if I pay on June 15th, then do I have to pay again on July 1st or July 15th?
July 15th. The validity period is 30 days from the day you make the payment. You can join anytime without waiting for a month to end.

What if I don’t like the training after making the payment. What is the refund policy?
You get a 7-day refund period from the day you have made the payment.

I am applying from a different country and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a transmission to mission.control@starlight-voyages.com.

Post-Registration Queries
Till when can I view the paid training videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased a subscription on April 21st, you will be able to watch all the past paid sessions in the period of April 21st to May 20th. But after May 21st, you will have to purchase the subscription again. Once the program is over and you have paid us $7192 (or 8 installments of $899) you will be able to watch the paid sessions until the launch window in 2026 closes.

Why is lifetime validity not provided?
Because mission parameters and technology are constantly evolving.

Where can I reach out in case of a doubt after a session?
You will have to fill a communication request form provided in your dashboard and our team will contact you for a 1-on-1 debriefing session.

If I join the program late, can I still ask about past modules?
Yes, just select the past module topic in the communication request form.

Certificate and Mission Placement Related Queries
What is the criteria to get the certificate of completion?
There are 2 criteria: You have to pay the entire fee of $7192, and you have to pass all the mission readiness assessments.

I am joining late. How can I pay the fee of the earlier months?
You will get a link to pay the fee for earlier months in your dashboard once you pay for the current month.

I have read that Mission Placement is a part of this program. What comes under Mission Placement?
This is to clarify that program completion does not mean a guarantee of a mission seat. We do not guarantee you a job or a spot on the Ares-IV crew. If you are joining this course just for a guaranteed flight, you will be disappointed. Here is what comes under mission placement consideration:

Advanced Portfolio Building

Psychological skills sessions

Sessions with veteran astronauts

Discussion on mission selection strategies
"""


In [4]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# tokenize
tokens = word_tokenize(document.lower())

In [6]:
# build vocab
vocab = {'<unk>':0}

for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token] = len(vocab)

vocab

{'<unk>': 0,
 'about': 1,
 'our': 2,
 'voyages': 3,
 'what': 4,
 'is': 5,
 'the': 6,
 'travel': 7,
 'package': 8,
 'for': 9,
 'martian': 10,
 'colony': 11,
 'initiative': 12,
 '(': 13,
 'mci': 14,
 '2025': 15,
 ')': 16,
 '?': 17,
 'uses': 18,
 'a': 19,
 'monthly': 20,
 'preparation': 21,
 'plan': 22,
 'where': 23,
 'you': 24,
 'make': 25,
 'payments': 26,
 'of': 27,
 '$': 28,
 '899/month': 29,
 'to': 30,
 'cover': 31,
 'training': 32,
 'and': 33,
 'logistics': 34,
 '.': 35,
 'total': 36,
 'duration': 37,
 'pre-flight': 38,
 'program': 39,
 '8': 40,
 'months': 41,
 'therefore': 42,
 ',': 43,
 'fee': 44,
 'becomes': 45,
 '899': 46,
 '*': 47,
 '=': 48,
 '7192': 49,
 'approx.': 50,
 'syllabus': 51,
 'astronaut': 52,
 'we': 53,
 'will': 54,
 'be': 55,
 'covering': 56,
 'following': 57,
 'modules': 58,
 ':': 59,
 'zero-gravity': 60,
 'acclimatization': 61,
 'exo-geology': 62,
 'planetary': 63,
 'science': 64,
 'hydroponics': 65,
 'sustainable': 66,
 'farming': 67,
 'sql': 68,
 'celestial': 6

In [7]:
len(vocab)

337

In [8]:
input_sentences = document.split('\n')

In [9]:
def text_to_indices(sentence, vocab):

  numerical_sentence = []

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence


In [10]:
input_numerical_sentences = []

for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))


In [11]:
len(input_numerical_sentences)

112

In [12]:
training_sequence = []
for sentence in input_numerical_sentences:

  for i in range(1, len(sentence)):
    training_sequence.append(sentence[:i+1])

In [13]:
len(training_sequence)

968

In [14]:
training_sequence[:5]

[[1, 2], [1, 2, 3], [4, 5], [4, 5, 6], [4, 5, 6, 7]]

In [15]:
len_list = []

for sequence in training_sequence:
  len_list.append(len(sequence))

max(len_list)

102

In [16]:
training_sequence[0]

[1, 2]

In [17]:
padded_training_sequence = []
for sequence in training_sequence:

  padded_training_sequence.append([0]*(max(len_list) - len(sequence)) + sequence)

In [18]:
len(padded_training_sequence[10])

102

In [19]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)

In [20]:
padded_training_sequence

tensor([[  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        [  0,   0,   0,  ...,   0,   4,   5],
        ...,
        [  0,   0,   0,  ..., 334, 109, 198],
        [  0,   0,   0,  ..., 109, 198, 335],
        [  0,   0,   0,  ..., 198, 335, 336]])

In [21]:
X = padded_training_sequence[:, :-1]
y = padded_training_sequence[:,-1]

In [22]:
X

tensor([[  0,   0,   0,  ...,   0,   0,   1],
        [  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   0,   0,   4],
        ...,
        [  0,   0,   0,  ...,   0, 334, 109],
        [  0,   0,   0,  ..., 334, 109, 198],
        [  0,   0,   0,  ..., 109, 198, 335]])

In [23]:
y

tensor([  2,   3,   5,   6,   7,   8,   9,   6,  10,  11,  12,  13,  14,  15,
         16,  17,  12,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
         29,  30,  31,  32,  33,  34,  35,   5,   6,  36,  37,  27,   6,  38,
         39,  17,  36,  37,  27,   6,  38,  32,  33,  21,   5,  40,  41,  35,
         42,  43,   6,  36,  21,  44,  45,  46,  47,  40,  48,  28,  49,  13,
          0,  35,  16,  35,   5,   6,  51,  27,   6,  52,  21,  39,  17,  54,
         55,  56,   6,  57,  58,  59,  61,  33,  63,  64,  33,  66,  67,   9,
         69,  70,  71,  73,  33,  74,  75,  77,  78,  80,  81,  83,  85,  27,
         86,  87,  88,  89,   6,  90,  51,  91,  92,  93,  59,  94,  32,   9,
          6,  95,  96,  55,  19,  97,  27,  98,  39,  17,  43, 100,  33, 101,
         87, 102, 103,  19,  97,  27,  98,  39, 104, 105, 106,  35,  98,   5,
        107, 108, 109, 110,  35, 111, 112, 113,  19, 114, 115,  17,  54, 112,
        116,  19, 117,  27, 118,  17,  43, 120,   2, 121, 122, 1

In [24]:
class CustomDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [25]:
dataset = CustomDataset(X,y)

In [26]:
len(dataset)

968

In [27]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [28]:
class LSTMModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 150, batch_first=True)
    self.fc = nn.Linear(150, vocab_size)

  def forward(self, x):
    embedded = self.embedding(x)
    intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output

In [29]:
model = LSTMModel(len(vocab))

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
model.to(device)

LSTMModel(
  (embedding): Embedding(337, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=337, bias=True)
)

In [32]:
epochs = 50
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [33]:
# training loop

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in dataloader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 176.3735
Epoch: 2, Loss: 155.8588
Epoch: 3, Loss: 143.8906
Epoch: 4, Loss: 132.4578
Epoch: 5, Loss: 121.9206
Epoch: 6, Loss: 109.8117
Epoch: 7, Loss: 100.0251
Epoch: 8, Loss: 91.1521
Epoch: 9, Loss: 80.8100
Epoch: 10, Loss: 73.1943
Epoch: 11, Loss: 65.6868
Epoch: 12, Loss: 59.1512
Epoch: 13, Loss: 52.2855
Epoch: 14, Loss: 46.1781
Epoch: 15, Loss: 41.6570
Epoch: 16, Loss: 36.3985
Epoch: 17, Loss: 32.2755
Epoch: 18, Loss: 29.2108
Epoch: 19, Loss: 25.8524
Epoch: 20, Loss: 22.9233
Epoch: 21, Loss: 20.4179
Epoch: 22, Loss: 18.2075
Epoch: 23, Loss: 16.6323
Epoch: 24, Loss: 15.1114
Epoch: 25, Loss: 13.7260
Epoch: 26, Loss: 12.5789
Epoch: 27, Loss: 11.6320
Epoch: 28, Loss: 10.7166
Epoch: 29, Loss: 10.1173
Epoch: 30, Loss: 9.3051
Epoch: 31, Loss: 8.6921
Epoch: 32, Loss: 8.1363
Epoch: 33, Loss: 8.1175
Epoch: 34, Loss: 7.4857
Epoch: 35, Loss: 7.1149
Epoch: 36, Loss: 6.6029
Epoch: 37, Loss: 6.4809
Epoch: 38, Loss: 6.0700
Epoch: 39, Loss: 5.8745
Epoch: 40, Loss: 5.7473
Epoch: 41, Lo

In [34]:
# prediction

def prediction(model, vocab, text):

  # tokenize
  tokenized_text = word_tokenize(text.lower())

  # text -> numerical indices
  numerical_text = text_to_indices(tokenized_text, vocab)

  # padding
  padded_text = torch.tensor([0] * (61 - len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)

  # send to model
  output = model(padded_text)

  # predicted index
  value, index = torch.max(output, dim=1)

  # merge with text
  return text + " " + list(vocab.keys())[index]



In [37]:
import time

num_tokens = 10
input_text = "Why is lifetime validity not provided?"

for i in range(num_tokens):
  output_text = prediction(model, vocab, input_text)
  print(output_text)
  input_text = output_text
  time.sleep(0.5)


Why is lifetime validity not provided? ?
Why is lifetime validity not provided? ? on
Why is lifetime validity not provided? ? on a
Why is lifetime validity not provided? ? on a partner
Why is lifetime validity not provided? ? on a partner site
Why is lifetime validity not provided? ? on a partner site or
Why is lifetime validity not provided? ? on a partner site or the
Why is lifetime validity not provided? ? on a partner site or the main
Why is lifetime validity not provided? ? on a partner site or the main website
Why is lifetime validity not provided? ? on a partner site or the main website ?


In [38]:
dataloader1 = DataLoader(dataset, batch_size=32, shuffle=False)

In [40]:
# Function to calculate accuracy
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients
        for batch_x, batch_y in dataloader1:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Get model predictions
            outputs = model(batch_x)

            # Get the predicted word indices
            _, predicted = torch.max(outputs, dim=1)

            # Compare with actual labels
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    return accuracy

# Compute accuracy
accuracy = calculate_accuracy(model, dataloader, device)
print(f"Model Accuracy: {accuracy:.2f}%")


Model Accuracy: 96.28%
